# Usage

## Clinical NER Sample

In [1]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/clinical_ner_sample/'

Corpus2GroupMethod = 'Dir' # TODO

Group2TextMethod   = 'file'

Text2SentMethod  = 're'

Sent2TokenMethod = 'iter'
TOKENLevel = 'char'

min_token_freq = 1

use_hyper = True

anno = 'annofile4text'
anno_keywords = {
    'ANNOIden': '.Entity',
    'anno_sep' : '\t', 
    'notZeroIndex' : 1, 
    'notRightOpen' : 0,
}


BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)



Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_r/46yh1st54y35_jbqhdz82lrh0000gp/T/jieba.cache


'Dir'
corpus/clinical_ner_sample/MEntity


Loading model cost 0.664 seconds.
Prefix dict has been built succesfully.


Total Num of All    Tokens 659
Total Num of Unique Tokens 245
CORPUS	it is Dumped into file: data/clinical_ner_sample/char/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
GROUP	it is Dumped into file: data/clinical_ner_sample/char/Pyramid/GROUP.p
GROUP	the length of it is   : 1
TEXT	it is Dumped into file: data/clinical_ner_sample/char/Pyramid/TEXT.p
TEXT	the length of it is   : 3
SENT	it is Dumped into file: data/clinical_ner_sample/char/Pyramid/SENT.p
SENT	the length of it is   : 21
TOKEN	it is Dumped into file: data/clinical_ner_sample/char/Pyramid/TOKEN.p
TOKEN	the length of it is   : 659
**************************************** 

pos-bioes	is Dumped into file: data/clinical_ner_sample/char/Vocab/pos-bioes.voc
pos-bioes	the length of it is   : 229
		Write to: data/clinical_ner_sample/char/Vocab/pos-bioes.tsv
annoE-bioes	is Dumped into file: data/clinical_ner_sample/char/Vocab/annoE-bioes.voc
annoE-bioes	the length of it is   : 21
		Write to: data/clinical_ner_sample/char/Vocab/an

## MedPOS Sample

In [3]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/medical_pos_sample/' # TODO

Corpus2GroupMethod = 'Dir' # TODO

Group2TextMethod   = 'file'

Text2SentMethod  = 're'

Sent2TokenMethod = 'iter'
TOKENLevel = 'char'

min_token_freq = 1

use_hyper = True

anno = 'annofile4sent'
anno_keywords = {
    'ANNOIden': '.UMLSTag',
    'anno_sep' : '\t', 
    'notZeroIndex' : 0, 
    'notRightOpen' : 0,
}


BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)




'Dir'
corpus/medical_pos_sample/batch1
10
patient4857-sent10.UMLSTag
2
patient5175-sent2.UMLSTag
corpus/medical_pos_sample/batch2
4
patient5218-sent4.UMLSTag
Total Num of All    Tokens 6460
Total Num of Unique Tokens 713
CORPUS	it is Dumped into file: data/medical_pos_sample/char/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
GROUP	it is Dumped into file: data/medical_pos_sample/char/Pyramid/GROUP.p
GROUP	the length of it is   : 2
TEXT	it is Dumped into file: data/medical_pos_sample/char/Pyramid/TEXT.p
TEXT	the length of it is   : 20
SENT	it is Dumped into file: data/medical_pos_sample/char/Pyramid/SENT.p
SENT	the length of it is   : 179
TOKEN	it is Dumped into file: data/medical_pos_sample/char/Pyramid/TOKEN.p
TOKEN	the length of it is   : 6460
**************************************** 

pos-bioes	is Dumped into file: data/medical_pos_sample/char/Vocab/pos-bioes.voc
pos-bioes	the length of it is   : 229
		Write to: data/medical_pos_sample/char/Vocab/pos-bioes.tsv
annoE-bioes	is Dump

## Boson

In [4]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/boson/'

Corpus2GroupMethod = '.txt'

Group2TextMethod   = 'line'

Text2SentMethod  = 're'

Sent2TokenMethod = 'iter'
TOKENLevel = 'char'

min_token_freq = 1

use_hyper = True

anno = 'anno_embed_in_text'
anno_keywords = {}

BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)


'File'
corpus/boson/bosonNER.txt
Total Num of All    Tokens 533491
Total Num of Unique Tokens 3865
CORPUS	it is Dumped into file: data/boson/char/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
GROUP	it is Dumped into file: data/boson/char/Pyramid/GROUP.p
GROUP	the length of it is   : 1
TEXT	it is Dumped into file: data/boson/char/Pyramid/TEXT.p
TEXT	the length of it is   : 1961
SENT	it is Dumped into file: data/boson/char/Pyramid/SENT.p
SENT	the length of it is   : 10214
TOKEN	it is Dumped into file: data/boson/char/Pyramid/TOKEN.p
TOKEN	the length of it is   : 533491
**************************************** 

pos-bioes	is Dumped into file: data/boson/char/Vocab/pos-bioes.voc
pos-bioes	the length of it is   : 229
		Write to: data/boson/char/Vocab/pos-bioes.tsv
annoE-bioes	is Dumped into file: data/boson/char/Vocab/annoE-bioes.voc
annoE-bioes	the length of it is   : 25
		Write to: data/boson/char/Vocab/annoE-bioes.tsv
annoE-bioes	is Dumped into file: data/boson/char/Vocab/token.voc
a

## ResumeCN

In [2]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/ResumeCN/'

Corpus2GroupMethod = '.bmes'

Group2TextMethod   = 'block'

Text2SentMethod  = 'whole'

Sent2TokenMethod = 'iter'
TOKENLevel = 'char'

min_token_freq = 1

use_hyper = True

anno = 'conll_block'
anno_keywords = {
    'anno_sep': ' ',
    'connector': '',
    'suffix': False,
}
BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_r/46yh1st54y35_jbqhdz82lrh0000gp/T/jieba.cache


'File'
corpus/ResumeCN/test.char.bmes


Loading model cost 0.697 seconds.
Prefix dict has been built succesfully.


corpus/ResumeCN/train.char.bmes
corpus/ResumeCN/dev.char.bmes
Total Num of All    Tokens 149123
Total Num of Unique Tokens 1865
CORPUS	it is Dumped into file: data/ResumeCN/char/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
GROUP	it is Dumped into file: data/ResumeCN/char/Pyramid/GROUP.p
GROUP	the length of it is   : 3
TEXT	it is Dumped into file: data/ResumeCN/char/Pyramid/TEXT.p
TEXT	the length of it is   : 4476
SENT	it is Dumped into file: data/ResumeCN/char/Pyramid/SENT.p
SENT	the length of it is   : 4476
TOKEN	it is Dumped into file: data/ResumeCN/char/Pyramid/TOKEN.p
TOKEN	the length of it is   : 149123
**************************************** 

pos-bioes	is Dumped into file: data/ResumeCN/char/Vocab/pos-bioes.voc
pos-bioes	the length of it is   : 229
		Write to: data/ResumeCN/char/Vocab/pos-bioes.tsv
annoE-bioes	is Dumped into file: data/ResumeCN/char/Vocab/annoE-bioes.voc
annoE-bioes	the length of it is   : 33
		Write to: data/ResumeCN/char/Vocab/annoE-bioes.tsv
annoE-bioes

## NewsCN

In [2]:
from pprint import pprint
from nlptext.base import BasicObject



CORPUSPath = 'corpus/NewsCN/'


Corpus2GroupMethod = '.char'

Group2TextMethod   = 'block'

Text2SentMethod  = 'whole'

Sent2TokenMethod = 'iter'
TOKENLevel = 'char'

min_token_freq = 1

use_hyper = True

anno = 'conll_block'
anno_keywords = {
    'anno_sep': ' ',
    'connector': '',
    'suffix': False,
}

BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)


/Users/floydluo/Environments/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_r/46yh1st54y35_jbqhdz82lrh0000gp/T/jieba.cache


'File'
corpus/NewsCN/demo.dev.char


Loading model cost 0.693 seconds.
Prefix dict has been built succesfully.


corpus/NewsCN/demo.train.char
corpus/NewsCN/demo.test.char
Total Num of All    Tokens 54635
Total Num of Unique Tokens 2329
CORPUS	it is Dumped into file: data/NewsCN/char/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
GROUP	it is Dumped into file: data/NewsCN/char/Pyramid/GROUP.p
GROUP	the length of it is   : 3
TEXT	it is Dumped into file: data/NewsCN/char/Pyramid/TEXT.p
TEXT	the length of it is   : 1129
SENT	it is Dumped into file: data/NewsCN/char/Pyramid/SENT.p
SENT	the length of it is   : 1129
TOKEN	it is Dumped into file: data/NewsCN/char/Pyramid/TOKEN.p
TOKEN	the length of it is   : 54635
**************************************** 

pos-bioes	is Dumped into file: data/NewsCN/char/Vocab/pos-bioes.voc
pos-bioes	the length of it is   : 229
		Write to: data/NewsCN/char/Vocab/pos-bioes.tsv
annoE-bioes	is Dumped into file: data/NewsCN/char/Vocab/annoE-bioes.voc
annoE-bioes	the length of it is   : 17
		Write to: data/NewsCN/char/Vocab/annoE-bioes.tsv
annoE-bioes	is Dumped into file: d

## CoNLL-2003

In [3]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/CoNLL-2003/'

Corpus2GroupMethod = '.openNLP'

Group2TextMethod   = 'block'

Text2SentMethod  = 'whole'

Sent2TokenMethod = ' '
TOKENLevel = 'word'

min_token_freq = 1

use_hyper = False

anno = 'conll_block'
anno_keywords = {
    'anno_sep': ' ',
    'connector': ' ', 
    'suffix': False,
    'change_tags': True, 
}

BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)


'File'
corpus/CoNLL-2003/eng.train.openNLP
corpus/CoNLL-2003/eng.testa.openNLP
corpus/CoNLL-2003/eng.testb.openNLP
Total Num of All    Tokens 254708
Total Num of Unique Tokens 27270
CORPUS	it is Dumped into file: data/CoNLL-2003/word/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
GROUP	it is Dumped into file: data/CoNLL-2003/word/Pyramid/GROUP.p
GROUP	the length of it is   : 3
TEXT	it is Dumped into file: data/CoNLL-2003/word/Pyramid/TEXT.p
TEXT	the length of it is   : 16477
SENT	it is Dumped into file: data/CoNLL-2003/word/Pyramid/SENT.p
SENT	the length of it is   : 16477
TOKEN	it is Dumped into file: data/CoNLL-2003/word/Pyramid/TOKEN.p
TOKEN	the length of it is   : 254708
**************************************** 

annoE-bioes	is Dumped into file: data/CoNLL-2003/word/Vocab/annoE-bioes.voc
annoE-bioes	the length of it is   : 17
		Write to: data/CoNLL-2003/word/Vocab/annoE-bioes.tsv
annoE-bioes	is Dumped into file: data/CoNLL-2003/word/Vocab/token.voc
annoE-bioes	the length of it i

## NLPBA2004

In [1]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/NLPBA2004/'

Corpus2GroupMethod = '.iob2'

Group2TextMethod   = 'block'

Text2SentMethod  = 'whole'

Sent2TokenMethod = ' '
TOKENLevel = 'word'

min_token_freq = 1

use_hyper = False

anno = 'conll_block'
anno_keywords = {
    'anno_sep': '\t',
    'connector': ' ', 
    'suffix': False,
}
BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)


'File'
corpus/NLPBA2004/sampletest2.iob2
corpus/NLPBA2004/Genia4ERtask2.iob2


/Users/floydluo/Environments/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


corpus/NLPBA2004/Genia4ERtask1.iob2
corpus/NLPBA2004/sampletest1.iob2
Total Num of All    Tokens 913580
Total Num of Unique Tokens 20663
CORPUS	it is Dumped into file: data/NLPBA2004/word/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
GROUP	it is Dumped into file: data/NLPBA2004/word/Pyramid/GROUP.p
GROUP	the length of it is   : 4
TEXT	it is Dumped into file: data/NLPBA2004/word/Pyramid/TEXT.p
TEXT	the length of it is   : 33492
SENT	it is Dumped into file: data/NLPBA2004/word/Pyramid/SENT.p
SENT	the length of it is   : 33492
TOKEN	it is Dumped into file: data/NLPBA2004/word/Pyramid/TOKEN.p
TOKEN	the length of it is   : 913580
**************************************** 

annoE-bioes	is Dumped into file: data/NLPBA2004/word/Vocab/annoE-bioes.voc
annoE-bioes	the length of it is   : 21
		Write to: data/NLPBA2004/word/Vocab/annoE-bioes.tsv
annoE-bioes	is Dumped into file: data/NLPBA2004/word/Vocab/token.voc
annoE-bioes	the length of it is   : 20663
		Write to: data/NLPBA2004/word/Vocab/tok

# Functions to Handle Labels

## strText and SSET

For each text, we must get a strText and SSET, which meet the following restriction.

Besides, we also need a strSents.

In [39]:

strText = '''结肠多发息肉。\n患中老年男性,慢性病程。 因“体检发现大肠多发息肉3月余”入院。查体:无阳性体征。'''

print(strText)

strAnnoText = '''标注文本名称:/Users/zhangling/Documents/新标的数据530/529李选-已检查/Entity/patient4378.txt\n标注文本字数统计:87\n多发息肉\t3\t6\t疾病\n慢性\t16\t17\t修饰\n多发息肉\t30\t33\t疾病\n3月余\t34\t36\t修饰\n无阳性体征\t44\t48\t不确定\n'''

# BIOES

sep = '\t'
SSETText = [sset.split('\t') for sset in strAnnoText.split('\n') if sep in sset]


notZeroIndex = 1 
for idx, sset in enumerate(SSETText):
    data = (sset[0], int(sset[1]) - notZeroIndex, int(sset[2]), sset[-1])
    SSETText[idx] = data


# check
for sset in SSETText:
    try:
        assert strText[sset[1]: sset[2]] == sset[0]
    except:
        print('strText:', strText[sset[1] : sset[2]])
        print('SSETText:', sset[0])
        
SSETText

结肠多发息肉。
患中老年男性,慢性病程。 因“体检发现大肠多发息肉3月余”入院。查体:无阳性体征。


[('多发息肉', 2, 6, '疾病'),
 ('慢性', 15, 17, '修饰'),
 ('多发息肉', 29, 33, '疾病'),
 ('3月余', 33, 36, '修饰'),
 ('无阳性体征', 43, 48, '不确定')]

## getCITText

In [40]:

strText = '''结肠多发息肉。\n患中老年男性,慢性病程。 因“体检发现大肠多发息肉3月余”入院。查体:无阳性体征。'''


SSETText = [ ('多发息肉', 2, 6, '疾病'),
             ('慢性', 15, 17, '修饰'),
             ('多发息肉', 29, 33, '疾病'),
             ('3月余', 33, 36, '修饰'),
             ('无阳性体征', 43, 48, '不确定')]

def getCITText(strText, SSETText, TOKENLevel='char'):
    len(SSETText) > 0 
    if TOKENLevel == 'char':
        for sset in SSETText:
            try:
                assert strText[sset[1]: sset[2]] == sset[0]
            except:
                print('strText:', strText[sset[1] : sset[2]])
                print('SSETText:', sset[0])
        CITAnnoText = []
        for sset in SSETText:
            # BIOES
            strAnno, s, e, tag = sset
            CIT = [[c, s + idx, tag+ '-I']  for idx, c in enumerate(strAnno)]
            CIT[-1][2] = tag + '-E'
            CIT[ 0][2] = tag + '-B'
            if len(CIT) == 1:
                CIT[0][2] = tag + '-S' 
            CITAnnoText.extend(CIT)

        # print(strAnnoText)
        CITText = [[char, idx, 'O'] for idx, char in enumerate(strText)]
        for citAnno in CITAnnoText:
            c, idx, t = citAnno
            assert CITText[idx][0] == c
            CITText[idx] = citAnno

    elif TOKENLevel == 'word':
        CITText = []
        for idx, sset in enumerate(SSETText):
            try:
                assert sset[0] == strText[idx]
            except:
                print(strText)[idx]
                print(sset[0])

            CITText.append(sset)
    return CITText


CITText = getCITText(strText, SSETText, TOKENLevel='char')
CITText

[['结', 0, 'O'],
 ['肠', 1, 'O'],
 ['多', 2, '疾病-B'],
 ['发', 3, '疾病-I'],
 ['息', 4, '疾病-I'],
 ['肉', 5, '疾病-E'],
 ['。', 6, 'O'],
 ['\n', 7, 'O'],
 ['患', 8, 'O'],
 ['中', 9, 'O'],
 ['老', 10, 'O'],
 ['年', 11, 'O'],
 ['男', 12, 'O'],
 ['性', 13, 'O'],
 [',', 14, 'O'],
 ['慢', 15, '修饰-B'],
 ['性', 16, '修饰-E'],
 ['病', 17, 'O'],
 ['程', 18, 'O'],
 ['。', 19, 'O'],
 [' ', 20, 'O'],
 ['因', 21, 'O'],
 ['“', 22, 'O'],
 ['体', 23, 'O'],
 ['检', 24, 'O'],
 ['发', 25, 'O'],
 ['现', 26, 'O'],
 ['大', 27, 'O'],
 ['肠', 28, 'O'],
 ['多', 29, '疾病-B'],
 ['发', 30, '疾病-I'],
 ['息', 31, '疾病-I'],
 ['肉', 32, '疾病-E'],
 ['3', 33, '修饰-B'],
 ['月', 34, '修饰-I'],
 ['余', 35, '修饰-E'],
 ['”', 36, 'O'],
 ['入', 37, 'O'],
 ['院', 38, 'O'],
 ['。', 39, 'O'],
 ['查', 40, 'O'],
 ['体', 41, 'O'],
 [':', 42, 'O'],
 ['无', 43, '不确定-B'],
 ['阳', 44, '不确定-I'],
 ['性', 45, '不确定-I'],
 ['体', 46, '不确定-I'],
 ['征', 47, '不确定-E'],
 ['。', 48, 'O']]

## getCITSents

In [2]:
strSents = ['结肠多发息肉。', '患中老年男性,慢性病程。', '因“体检发现大肠多发息肉3月余”入院。', '查体:无阳性体征。']

CITText  =  [['结', 0, 'O'],
             ['肠', 1, 'O'],
             ['多', 2, '疾病-B'],
             ['发', 3, '疾病-I'],
             ['息', 4, '疾病-I'],
             ['肉', 5, '疾病-E'],
             ['。', 6, 'O'],
             ['\n', 7, 'O'],
             ['患', 8, 'O'],
             ['中', 9, 'O'],
             ['老', 10, 'O'],
             ['年', 11, 'O'],
             ['男', 12, 'O'],
             ['性', 13, 'O'],
             [',', 14, 'O'],
             ['慢', 15, '修饰-B'],
             ['性', 16, '修饰-E'],
             ['病', 17, 'O'],
             ['程', 18, 'O'],
             ['。', 19, 'O'],
             [' ', 20, 'O'],
             ['因', 21, 'O'],
             ['“', 22, 'O'],
             ['体', 23, 'O'],
             ['检', 24, 'O'],
             ['发', 25, 'O'],
             ['现', 26, 'O'],
             ['大', 27, 'O'],
             ['肠', 28, 'O'],
             ['多', 29, '疾病-B'],
             ['发', 30, '疾病-I'],
             ['息', 31, '疾病-I'],
             ['肉', 32, '疾病-E'],
             ['3', 33, '修饰-B'],
             ['月', 34, '修饰-I'],
             ['余', 35, '修饰-E'],
             ['”', 36, 'O'],
             ['入', 37, 'O'],
             ['院', 38, 'O'],
             ['。', 39, 'O'],
             ['查', 40, 'O'],
             ['体', 41, 'O'],
             [':', 42, 'O'],
             ['无', 43, '不确定-B'],
             ['阳', 44, '不确定-I'],
             ['性', 45, '不确定-I'],
             ['体', 46, '不确定-I'],
             ['征', 47, '不确定-E'],
             ['。', 48, 'O']]

def getCITSents(strSents, CITText):
    lenLastSent = 0
    collapse    = 0 # don't need to move 
    CITSents = []
    for strSent in strSents:
        CITSent = []
        for sentTokenIdx, c in enumerate(strSent):
            # sentTokenIdx = txtTokenIdx - lenLastSent - collapse
            txtTokenIdx = sentTokenIdx + lenLastSent + collapse
            cT, _, tT = CITText[txtTokenIdx]
            while c != cT and c != ' ':
                collapse = collapse + 1
                txtTokenIdx = sentTokenIdx + lenLastSent + collapse
                cT, _, tT = CITText[txtTokenIdx]
            CITSent.append([c,sentTokenIdx, tT])
        lenLastSent = lenLastSent + len(strSent)
        CITSents.append(CITSent)
    # CITSents
    # Here we get CITSents  
    return CITSents
       
    
CITSents = getCITSents(strSents, CITText)
CITSents

[[['结', 0, 'O'],
  ['肠', 1, 'O'],
  ['多', 2, '疾病-B'],
  ['发', 3, '疾病-I'],
  ['息', 4, '疾病-I'],
  ['肉', 5, '疾病-E'],
  ['。', 6, 'O']],
 [['患', 0, 'O'],
  ['中', 1, 'O'],
  ['老', 2, 'O'],
  ['年', 3, 'O'],
  ['男', 4, 'O'],
  ['性', 5, 'O'],
  [',', 6, 'O'],
  ['慢', 7, '修饰-B'],
  ['性', 8, '修饰-E'],
  ['病', 9, 'O'],
  ['程', 10, 'O'],
  ['。', 11, 'O']],
 [['因', 0, 'O'],
  ['“', 1, 'O'],
  ['体', 2, 'O'],
  ['检', 3, 'O'],
  ['发', 4, 'O'],
  ['现', 5, 'O'],
  ['大', 6, 'O'],
  ['肠', 7, 'O'],
  ['多', 8, '疾病-B'],
  ['发', 9, '疾病-I'],
  ['息', 10, '疾病-I'],
  ['肉', 11, '疾病-E'],
  ['3', 12, '修饰-B'],
  ['月', 13, '修饰-I'],
  ['余', 14, '修饰-E'],
  ['”', 15, 'O'],
  ['入', 16, 'O'],
  ['院', 17, 'O'],
  ['。', 18, 'O']],
 [['查', 0, 'O'],
  ['体', 1, 'O'],
  [':', 2, 'O'],
  ['无', 3, '不确定-B'],
  ['阳', 4, '不确定-I'],
  ['性', 5, '不确定-I'],
  ['体', 6, '不确定-I'],
  ['征', 7, '不确定-E'],
  ['。', 8, 'O']]]

## getSSET_from_CIT

In [6]:
CITSent = CITSents[0]
print(CITSent)
tag_seq = [i[-1] for i in CITSent]
tag_seq

[['结', 0, 'O'], ['肠', 1, 'O'], ['多', 2, '疾病-B'], ['发', 3, '疾病-I'], ['息', 4, '疾病-I'], ['肉', 5, '疾病-E'], ['。', 6, 'O']]


['O', 'O', '疾病-B', '疾病-I', '疾病-I', '疾病-E', 'O']

In [ ]:
def getSSET_from_CIT(orig_seq = None, tag_seq = None, CIT = None, tag_seq_tagScheme = 'BIO', join_char = ''):
    # orig_seq is sentence without start or end
    # tag_seq may have start or end
        
    tagScheme = tag_seq_tagScheme
    if tagScheme == 'BIOES':
        tag_seq = [i.replace('-S', '-B').replace('-E', '-I') for i in tag_seq]
    elif tagScheme == 'BIOE':
        tag_seq = [i.replace('-E', '-I') for i in tag_seq]
    elif tagScheme == 'BIOS':
        tag_seq = [i.replace('-S', '-B') for i in tag_seq]
    elif tagScheme == 'BIO':
        pass
    else:
        print('The tagScheme', tagScheme, 'is not supported yet...')
    
    if not CIT:
        # use BIO tagScheme
        CIT = list(zip(orig_seq, range(len(orig_seq)), tag_seq))
    taggedCIT = [cit for cit in CIT if cit[2]!= 'O']
    
    startIdx = [idx for idx in range(len(taggedCIT)) if taggedCIT[idx][2][-2:] == '-B']
    startIdx.append(len(taggedCIT))

    entitiesList = []
    for i in range(len(startIdx)-1):
        entityAtom = taggedCIT[startIdx[i]: startIdx[i+1]]
        string = join_char.join([cit[0] for cit in entityAtom])
        start, end = entityAtom[0][1], entityAtom[-1][1] + 1
        tag = entityAtom[0][2].split('-')[0]
        entitiesList.append((string, start, end, tag))
    return entitiesList

# Get strText, strSents, and SSET